Загрузка

In [17]:
import pandas as pd
import numpy as np
import re
import string
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report

# Загрузка данных
train_data = pd.read_csv('train.csv', sep=';', encoding='utf-8')
test_data = pd.read_csv('test.csv', sep=';', encoding='utf-8')# Количество строк в тестовом наборе данных перед обработкой
initial_test_length = len(test_data)
print(f"\nКоличество строк в test_data до обработки: {initial_test_length}")

# Проверка на дубликаты и их удаление
print(f"Количество дубликатов в train_data до удаления: {train_data.duplicated().sum()}")
train_data = train_data.drop_duplicates()
print(f"Количество дубликатов в train_data после удаления: {train_data.duplicated().sum()}")

# Проверка на наличие пропущенных значений
print("\nИнформация о пропущенных значениях в train_data:")
print(train_data.isnull().sum())

print("\nИнформация о пропущенных значениях в test_data:")
print(test_data.isnull().sum())


# Если есть пропущенные значения в колонке 'text', удалите соответствующие строки
test_data_na = test_data[test_data['text'].isna()]
test_data = test_data.dropna(subset=['text'])

# Количество удаленных строк
removed_rows = initial_test_length - len(test_data)
print(f"Количество строк, удаленных из test_data: {removed_rows}")
print("\nУдаленные строки из test_data из-за пропущенных значений в колонке 'text':")
print(test_data_na)

# Проверка данных после обработки
print(f"\nКоличество строк в train_data после обработки: {len(train_data)}")
print(f"Количество строк в test_data после обработки: {len(test_data)}")

# Сохранение очищенных данных (если необходимо)
train_data.to_csv('cleaned_train.csv', sep=';', index=False, encoding='utf-8')
test_data.to_csv('cleaned_test.csv', sep=';', index=False, encoding='utf-8')

# Очистка текста
def clean_text(text):
    text = text.lower()
    text = re.sub(r'[^\w\s]', ' ', text)  # Оставить только буквы и пробелы
    text = re.sub(r'\d+', '', text)       # Удалить цифры
    text = re.sub(r'\s+', ' ', text).strip()
    return text

train_data['text'] = train_data['text'].apply(clean_text)
test_data['text'] = test_data['text'].apply(clean_text)

# Создание целевых меток
companies = ['sber', 'vtb', 'gazprom', 'alfabank', 'raiffeisen', 'rshb','company']
for company in companies:
    train_data[f'{company}_label'] = train_data[company].apply(lambda x: -1 if x == -1 else (1 if x == 1 else (0 if x == 0 else np.nan)))

# Удаление строк без меток
train_data.dropna(subset=[f'{company}_label' for company in companies], how='all', inplace=True)

# Обучение моделей для каждой компании
models = {}
for company in companies:
    company_data = train_data.dropna(subset=[f'{company}_label'])
    X = company_data['text']
    y = company_data[f'{company}_label']
    
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
    
    pipeline = Pipeline([
        ('tfidf', TfidfVectorizer(max_features=10000)),
        ('clf', LogisticRegression(max_iter=1000))
    ])
    
    pipeline.fit(X_train, y_train)
    y_pred = pipeline.predict(X_val)
    print(f"Classification report for {company}:\n", classification_report(y_val, y_pred))
    
    models[company] = pipeline

# Создание предсказаний для тестовых данных
predictions = []
for _, row in test_data.iterrows():
    row_preds = {'id': row['id']}
    text = row['text']
    for company in companies:
        pred = models[company].predict([text])[0]
        row_preds[f'{company}_n'] = 1 if pred == -1 else 0
        row_preds[f'{company}_0'] = 1 if pred == 0 else 0
        row_preds[f'{company}_p'] = 1 if pred == 1 else 0
    predictions.append(row_preds)

# Преобразование предсказаний в DataFrame и сохранение в файл
submission = pd.DataFrame(predictions)
submission.to_csv('submission.csv', index=False)

# Печать нескольких примеров предсказаний
print(submission.head())



Количество строк в test_data до обработки: 4779
Количество дубликатов в train_data до удаления: 0
Количество дубликатов в train_data после удаления: 0

Информация о пропущенных значениях в train_data:
id               0
text             0
sber          3817
vtb           5683
gazprom       6757
alfabank      6507
raiffeisen    6888
rshb          6693
company       6418
dtype: int64

Информация о пропущенных значениях в test_data:
id      0
text    0
dtype: int64
Количество строк, удаленных из test_data: 0

Удаленные строки из test_data из-за пропущенных значений в колонке 'text':
Empty DataFrame
Columns: [id, text]
Index: []

Количество строк в train_data после обработки: 7161
Количество строк в test_data после обработки: 4779
Classification report for sber:
               precision    recall  f1-score   support

        -1.0       0.85      0.63      0.72       133
         0.0       0.80      0.95      0.87       409
         1.0       0.84      0.56      0.67       127

    accurac

c:\Users\Stasya\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Stasya\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Stasya\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average,

     id  sber_n  sber_0  sber_p  vtb_n  vtb_0  vtb_p  gazprom_n  gazprom_0  \
0  2807       0       1       0      0      1      0          0          1   
1  5231       0       1       0      0      1      0          0          1   
2  9155       1       0       0      1      0      0          1          0   
3  2525       0       1       0      0      1      0          0          1   
4  7702       0       1       0      0      1      0          0          1   

   gazprom_p  ...  alfabank_p  raiffeisen_n  raiffeisen_0  raiffeisen_p  \
0          0  ...           0             0             1             0   
1          0  ...           0             0             1             0   
2          0  ...           0             0             1             0   
3          0  ...           0             0             1             0   
4          0  ...           0             0             1             0   

   rshb_n  rshb_0  rshb_p  company_n  company_0  company_p  
0       0       1  

In [11]:
# Загрузка образца файла для сравнения
sample_submission = pd.read_csv('submission.csv')
print("Образец файла submission.csv:")
print(sample_submission.head())
print(f"Количество строк в submission: {len(sample_submission)}")


Образец файла submission.csv:
     id  sber_n  sber_0  sber_p  vtb_n  vtb_0  vtb_p  gazprom_n  gazprom_0  \
0  2807       0       1       0      0      1      0          0          1   
1  5231       0       1       0      0      1      0          0          1   
2  9155       1       0       0      1      0      0          1          0   
3  2525       0       1       0      0      1      0          0          1   
4  7702       0       1       0      0      1      0          0          1   

   gazprom_p  ...  alfabank_p  raiffeisen_n  raiffeisen_0  raiffeisen_p  \
0          0  ...           0             0             1             0   
1          0  ...           0             0             1             0   
2          0  ...           0             0             1             0   
3          0  ...           0             0             1             0   
4          0  ...           0             0             1             0   

   rshb_n  rshb_0  rshb_p  company_n  company_0  c

In [12]:
# Загрузка образца файла для сравнения
sample_submission = pd.read_csv('sample_submission.csv')
print("Образец файла samle_submission.csv:")
print(sample_submission.head())
print(f"Количество строк в samle_submission: {len(sample_submission)}")


Образец файла samle_submission.csv:
     id  sber_n  sber_0  sber_p  vtb_n  vtb_0  vtb_p  gazprom_n  gazprom_0  \
0  2807       0       0       0      1      0      0          0          0   
1  5231       0       0       0      0      1      0          0          0   
2  9155       0       0       0      0      0      0          0          0   
3  2525       0       0       0      0      0      0          0          0   
4  7702       0       0       0      0      0      0          0          0   

   gazprom_p  ...  alfabank_p  raiffeisen_n  raiffeisen_0  raiffeisen_p  \
0          0  ...           0             0             0             0   
1          0  ...           0             0             0             0   
2          0  ...           0             0             0             0   
3          0  ...           0             0             1             0   
4          0  ...           0             0             0             0   

   rshb_n  rshb_0  rshb_p  company_n  compan

In [13]:
# Загрузка файла sample_submission.csv
sample_submission_data = pd.read_csv('sample_submission.csv')

# Загрузка файла submission.csv
submission_data = pd.read_csv('submission.csv')

# Преобразование id в множества для обоих файлов
sample_ids = set(sample_submission_data['id'])
submission_ids = set(submission_data['id'])

# Определение отсутствующих id в submission.csv
missing_ids = sample_ids - submission_ids

print("Отсутствующие id в файле submission.csv:", missing_ids)


Отсутствующие id в файле submission.csv: {6499, 5131, 16780, 2540, 2770, 14995, 5524, 2517, 6485}


In [14]:
missing_ids = set([6499, 5131, 16780, 2540, 2770, 14995, 5524, 2517, 6485]) - set(test_data['id'])
print("Отсутствующие id в тестовых данных:", missing_ids)


Отсутствующие id в тестовых данных: {6499, 5131, 16780, 2540, 2770, 14995, 5524, 2517, 6485}


In [15]:
unique_submission_ids = set(submission['id'])
print("Количество уникальных id в предсказаниях:", len(unique_submission_ids))


Количество уникальных id в предсказаниях: 4779


In [16]:
duplicate_submission_ids = submission[submission.duplicated(subset=['id'], keep=False)]['id'].tolist()
print("Дубликаты id в предсказаниях:", duplicate_submission_ids)


Дубликаты id в предсказаниях: []
